In [21]:
#Die ganzen packages die für den versuch verwendet werden

import numpy as np #numpy ist ein package mit allen mathematischen ausdrücken/funktionen/operationen/etc. 
from uncertainties import ufloat #Uncertainties ist für das automatische berrechnen von unsicherheiten
from uncertainties.umath import * #Importieren von rechnen mit uncertainties (addition/multiplikation/etc.)
from uncertainties import unumpy as unp #kann arrays von unsicherheiten verstehen (also eine liste, statt jede zahl einzeln einzutippen)
from IPython.display import Latex, HTML, Math, display # Um ergebnisse schöner anzuzeigen. display() "ersetzt" quasi print(), und lässt latex fromatierung zu

#funktion für mittwlert mit fehler defnieren (um später ein uarray mit unsicherheiten direkt zu einem mittelwert zu machen)
def mean(values):
    nom = unp.nominal_values(values)
    std_mean = np.std(nom, ddof=1) / np.sqrt(len(nom))
    return ufloat(np.mean(nom), std_mean)


unsicherheit = 1 #mm?   #Ich nehme mal an das die gemessene unsicherheit immer gleich bleibt, also brauchen wir die nur einmal

# Geometrische Optik

## Brennweiten von Linsen

### Versuchsaufbau und Durchführung

Auf einer optischen Bank werden Lichtquelle, Linse(n), Gegenstand und Schirm platziert. Dabei wird darauf geachtet, dass die optiche Achse mittig durch die Linse(n) läuft.

Zur Bestimmung der Brennweite einer Konvexlinse wird zuerst die Bildweite bei eienr fixen Gegenstandsweite gemessen, indem der Schirm bewegt wird, bis das Bild scharf ist. Hierbei wird eine Messreihe von mindestens fünf Werten angefertigt und die Fehler der ermittelten Größe berechnet.
Anschließend wird die Brennweite mit dem Besselverfahren ermittelt. Hier werden fünf verschiedene Abstände $e$ zwischen Gegenstand und Schirm eingestellt, wobei hier darauf geachtet werden muss, dass dieser Abstand mindestens viermal so groß wie die Brennweite $f$ der Linse ist. Es wird jeweils der Abstand $d$ zwischen den beiden Punkten der Linse bestimmt, an denen das Bild scharf ist, und so die Brennweite ermittelt.

Nun wird die Brennweite einer Konkavlinse ermittelt. Da diese kein reelles Bild zeigt, muss mit Hilfe einer Konvexlinse gearbeitet werden (siehe Skizze Versuchsaufbau). Zuerst wird mithilfe der Konvexlinse ein verkleinertes Bild erzeugt. Dieses dient der Konkavlinse als Gegenstand. Sie wird nun zwischen Schirm und Konvexlinse (mit Abstand $d$) eingebracht und der Schirm soweit verschoben, bis ein reelles, scharfes Bild zu sehen ist. Nun kann die Brennweite der Konkavlinse berechnet werden. 

![Skizze](Skizze.png)

Aus den jeweils berechneten Brennweiten der Linsen wird zuletzt noch die Dioptrie $D$ bestimmt.

#### Wichtige Formeln

* Linsengleichung: $\frac{1}{f}=\frac{1}{b}+\frac{1}{g}$
* Besselverfahren: $f=\frac{1}{4}(e-\frac{d^2}{e})$
* Gegenstandsweite $g_2$ bei Bestimmung der Brennweite der Konkavlinse: $g_2=-(b_1-d)$
* Dioptrie: $D=\frac{1}{f}$

#### Verwendete Geräte und Unsicherheiten

* Optische Bank: 1 mm ? (Auflösung)

### Ergebnisse

In [22]:
#Konvexlinse Brennweite messen


####################################################################################
#Punkt a,

#werte liste für die gegenstands/bildweite (entfernung von linse) g/b:
#üssen nur 5 min. sein

g_1_werte = 300   #mm #gegenstandsweite bleibt konstant
b_1_werte = np.array([995, 983, 992, 987, 990, 990, 991, 998, 999, 991])   #mm

#unsicherheitsabelle mit einzelnen werten erstellen:

g = ufloat(g_1_werte, unsicherheit)
b = ufloat(1300, 1) - unp.uarray(b_1_werte, unsicherheit) #position von schirm von der position der linse abziehen um die länge b zu bekommen

#formel für brennweite f:

f = 1/(1/b+1/g)       #mm

#f ist ein wert mit unsicherheit f = f_nom +- f_std
f_nom = unp.nominal_values(f)       #nominal-werte von f (also nur die werte)
f_std = unp.std_devs(f)             #standard deviation von f (also nur unsicherheit)

#-----------------------------------------------------------------------------------
#ab hier nur noch um die daten schon zu darzustellen:

#Latex tabelle definieren
table = r"""$$
\begin{array}{c|c}
\textbf{} & \textbf{Brennweite} ~f~[\mathrm{mm}] \\ \hline
"""
#für jedes i-te paar (nom,std) in der numerierten liste [(1,0.1),(2,0.1),(3,0.1)...] fügen wir der vorher definierten tabelle einen neuen eintrag ein:
#(table += rf"{i-ter eintrag} & wertepaar"). Wenn keine i mehr übrig sind hört der loop mit table += r"\end{array}" auf.
for i, (nom, std) in enumerate(zip(f_nom, f_std)):
    table += rf"{i} & ({nom:.2f} \pm {std:.2f}) \\ "
table += rf"\\ {{mittel}} & ({unp.nominal_values(mean(f)):.0f} \pm {1})" #bevor der loop endet wird noch in die letzte zeile der mittelwert dazugegeben
table += r"\end{array}$$"

#Tabelle anzeigen lassen (printen)
display(Latex(table))

#Zeigt den berrechneten wert 1/mittelwert an und konvertiert von mm in m (1/1000)
display(Latex(rf"$${{\frac{1}{{f}}}} = D = ({1/(ufloat((np.mean(unp.nominal_values(f))),1)/1000):.2f}~\mathrm){{\frac{1}{{m}}}}$$"))
#anhebung der Unsicherheit aufgrund menschlichen Fehlers und schwer erkennbarer Bilder 
#-----------------------------------------------------------------------------------

####################################################################################








####################################################################################
#Punkt b, Besselverfahren

#werte liste für e (gegenstand-schirm abstand) und d = d_2 - d_1 (verschiebung/symmetrische linsenposition):


e =   ufloat(1600, 1) - unp.uarray([900, 850, 800, 750, 700], unsicherheit) #mm  #position von objekt (fix) minus position von schirm ist e
d_1 =  unp.uarray([1133, 1072, 1015, 960, 903], unsicherheit) #mm  #erste linsen position
d_2 = unp.uarray([1379, 1391, 1397, 1405, 1410], unsicherheit) #mm #zweite linsen postion

d = d_2-d_1 #linsen postionen voneinander abziehen um total zu bekommen.

#formel für brennweite f:

f_1 = (1/4)*(e-((d**2)/e))       #mm

f_nom_1 = unp.nominal_values(f_1) 
f_std_1 = unp.std_devs(f_1)

display(Latex(r"$$\mathrm{\textbf{Besselverfahren}}$$"))
table = r"""$$
\begin{array}{c|c}
\textbf{} & \textbf{Brennweite} ~f~[\mathrm{mm}] \\ \hline
"""
for i, (nom_1, std_1) in enumerate(zip(f_nom_1, f_std_1)):
    table += rf"{i} & ({nom_1:.2f} \pm {std_1:.2f}) \\ "
table += rf"\\ {{mittel}} & ({unp.nominal_values(mean(f_1)):.0f} \pm {1})"
table += r"\end{array}$$"

display(Latex(table))
#Zeigt den berrechneten wert 1/mittelwert an und konvertiert von mm in m (1/1000)
display(Latex(rf"$${{\frac{1}{{f}}}} = D = ({1/(ufloat(np.mean((unp.nominal_values(f_1))),1)/1000):.2f}~\mathrm){{\frac{1}{{m}}}}$$"))
#!!!!!!!!!!!!!UNSICHERHEIT IST VON UNS EINGETRAGEN WORDEN
#anhebung der Unsicherheit aufgrund menschlichen Fehlers und schwer erkennbarer Bilder 
####################################################################################



<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [23]:
#Konkavlinse Brennweite messen

#Konvexlinse daten #2 mal durchgeführt den versuch
g_1_werte = np.array([306, 300])   #mm
b_1_werte = np.array([294, 310])   #mm

#Konkavlinse daten
d_werte = np.array([1306-1263, 43]) #Abstand der beiden linsen
b_2_werte = np.array([1263-620, 1257-657]) #abstand zum schirm

d = unp.uarray(d_werte, unsicherheit)
b_2 = unp.uarray(b_2_werte, unsicherheit)

g_1 = unp.uarray(g_1_werte, unsicherheit) 
b_1 = unp.uarray(b_1_werte, unsicherheit)

g_2 = -(b_1-d)

f = 1/(1/b_2+1/g_2)       #mm

f_nom = unp.nominal_values(f)
f_std = unp.std_devs(f)

#-----------------------------------------------------------------------------------
table = r"""$$
\begin{array}{c|c}
\textbf{} & \textbf{Brennweite} ~f~[\mathrm{mm?}] \\ \hline
"""
for i, (nom, std) in enumerate(zip(f_nom, f_std)):
    table += rf"{i} & ({nom:.2f} \pm {std:.2f}) \\ "
table += rf"\\ {{mittel}} & ({unp.nominal_values(mean(f)):.2f} \pm {unp.std_devs(mean(f)):.2f})"
table += r"\end{array}$$"

display(Latex(table))
#Die tausend ist um von 1/mm in 1/m umzuwandeln
display(Latex(rf"$${{\frac{1}{{f}}}} = D = ({1/(ufloat(np.mean((unp.nominal_values(f))),1)/1000):.2f}~\mathrm){{\frac{1}{{m}}}}$$"))
        
#!!!!!!!!!!!!!!!von mir gerunderter wert!!!!!!!!!!!!!!!

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

### Diskussion


## Linsenfehler

### Versuchsaufbau und Durchführung

Zur Bestimmung der sphärischen Abberation eienr Konvexlinse wird ähnlich vorgegangen wie bei der Bestimmung ihrer Brennweite, mit dem Unterschied, dass Blenden verwendet werden um abwechselnd achsennahe und achsenferne Strahlen auszublenden. Es wird erneut eine Messreihe von mindestens fünf Messungen angefertigt, die Brennweiten für achsenferne uns achsennahe Strahlen mithilfe der Linsengleichung berechnet und deren Unsicherheit angegeben.

Um die chromatische Abberation zu bestimmen werden Farbfilter in den Farben rot, grün und blau verwendet. Vor dem Einsetzen der Farbfilter wird der Abstand zwischen Lichtquelle und Gegenstand erhöht und ein möglichst paralleles Lichtbündel erzeugt. Hierzu kann auch die Blende zum Ausblenden der achsenfernen Strahlen verwendet werden. Anschließend werden erneut die Bildweiten bestimmt und die Brennweiten mitsamt Unsicherheiten berechnet. Auch hier wird eine Messreihe von jeweils mindestens fünf Messungen angefertigt.  

#### Wichtige Formeln

* Linsengleichung: $\frac{1}{f}=\frac{1}{b}+\frac{1}{g}$

#### Verwendete Geräte und Unsicherheiten

* Optische Bank: 1 mm ? (Auflösung)

### Ergebnisse

In [24]:
#Sphärische abberration

b_n = ufloat(1300, 1) - unp.uarray([982, 984, 981, 984, 983], unsicherheit) #mm           #Achsennahe Blende
b_f = ufloat(1300, 1) - unp.uarray([1024, 1030, 1024, 1017, 1021], unsicherheit) #mm      #Achsenferne Blende

table = r"""$$
\begin{array}{cc}
\\ \textbf{Ferne Bildweite} ~b_f~[\mathrm{mm}] & \textbf{Nahe Bildweite} ~b_n~[\mathrm{mm}]\\ \hline \\
"""
for i, (bn, bf) in enumerate(zip(b_f, b_n), start=1):
    table += rf"({unp.nominal_values(bn):.2f} \pm {unp.std_devs(bn):.2f}) & "
    table += rf"({unp.nominal_values(bf):.2f} \pm {unp.std_devs(bf):.2f}) \\ "

#hier werden die mittelwerte in der tabelle angezeigt 
table += r"\\ \hline \\"
table += rf"\bar{{b_f}}=({unp.nominal_values(mean(b_f)):.2f} \pm {unp.std_devs(mean(b_f)):.2f}) & "
table += rf"\bar{{b_n}}=({unp.nominal_values(mean(b_n)):.2f} \pm {unp.std_devs(mean(b_n)):.2f}) \\ "

#hier werden die letzendlichen werte in der tabelle angezeigt 
# 
# !!!!!!!!!!(die unsicherheit wurden auf 5mm gerundet)!!!!!!!!!!
table += rf"\\ b_f = ({unp.nominal_values(mean(b_f)):.0f} \pm  5) &"
table += rf" b_n = ({unp.nominal_values(mean(b_n)):.0f} \pm  5)\\"
table += r"\end{array}$$"

display(Latex(table))



<IPython.core.display.Latex object>

In [25]:
#Chromatische Aberration


b_r = ufloat(750, 1) - unp.uarray([562 , 562, 562, 563, 562, 563], unsicherheit) #mm
b_g = ufloat(750, 1) - unp.uarray([560 , 560, 559, 561, 561, 562], unsicherheit) #mm
b_b = ufloat(750, 1) - unp.uarray([562 ,563 , 563, 562, 561, 564], unsicherheit) #mm


table = r"""
$$
\begin{array}{ccc}
\\ \textbf{Rote Bildweite}~b_r~[\mathrm{mm}] & \textbf{Grüne Bildweite}~b_g~[\mathrm{mm}] & \textbf{Blaue Bildweite}~b_b~[\mathrm{mm}] \\ \\ \hline \\
"""
for i, (br, bg, bb) in enumerate(zip(b_r, b_g, b_b), start=1):
    table += rf"({unp.nominal_values(br):.2f} \pm {unp.std_devs(br):.2f}) & "
    table += rf"({unp.nominal_values(bg):.2f} \pm {unp.std_devs(bg):.2f}) & "
    table += rf"({unp.nominal_values(bb):.2f} \pm {unp.std_devs(bb):.2f}) \\"

#hier werden die mittelwerte in der tabelle angezeigt
table += r"\\ \hline \\"
table += rf"({unp.nominal_values(mean(b_r)):.2f} \pm {unp.std_devs(mean(b_r)):.2f}) & "
table += rf"({unp.nominal_values(mean(b_g)):.2f} \pm {unp.std_devs(mean(b_g)):.2f}) & "
table += rf"({unp.nominal_values(mean(b_b)):.2f} \pm {unp.std_devs(mean(b_b)):.2f}) \\ "

#hier werden die letzendlichen werte in der tabelle angezeigt 
# 
# !!!!!!!!!!(die unsicherheit wurden auf 5mm gerundet)!!!!!!!!!!
table += rf"\\ b_r = ({unp.nominal_values(mean(b_r)):.0f} \pm 5) & "
table += rf" b_g =({unp.nominal_values(mean(b_g)):.0f} \pm 5) & "
table += rf" b_b =({unp.nominal_values(mean(b_b)):.0f} \pm 5) \\ "
table += r"\end{array}$$"

display(Latex(table))

<IPython.core.display.Latex object>

### Diskussion

